<a href="https://colab.research.google.com/github/Mondin0/data-eng/blob/main/CeL_Almacenamiento_Delta_lake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Lakehouse

Antes de hablar sobre el tema principal de esta notebook, es importante entender una arquitectura de almacenamiento de datos relativamente reciente: Data Lakehouse.

Hasta ahora, hemos aprendido sobre dos arquitecturas principales: Data Warehouse y Data Lake. Cada una tiene sus ventajas, desventajas y escenarios ideales de aplicación. En algunos casos, es posible combinar ambas en una solución híbrida, pero eso no significa que sea un Data Lakehouse.

Cada arquitectura tiene aplicaciones específicas:
- Data Lake se usa principalmente en soluciones de Inteligencia Artificial, Machine Learning y Ciencia de Datos (es decir, analítica predictiva y prescriptiva).
- Data Warehouse es ideal para reportes e inteligencia de negocios (analítica descriptiva y diagnóstica).

Ahora bien ¿qué es un Data lakehouse?
Primero, miremos esta imagen sobre la evolución de arquitecturas de datos: [Evolución de arquitecturas](https://media.licdn.com/dms/image/v2/D5612AQEMvf-lL0b64Q/article-cover_image-shrink_720_1280/article-cover_image-shrink_720_1280/0/1676637608474?e=1746662400&v=beta&t=mrqR3fsTxIgRji1fchZKnlcaHJFjnSM77hoLSw9wBYk)

A partir de esta ilustración, podemos decir que Data Lakehouse:
- es un Data Lake, ya que mantiene el almacenamiento basado en archivos en lugar de tablas, como lo hace Data Warehouse.
- cuenta con una capa transaccional, que permite gestionar los archivos como si fueran tablas de base de datos, garantizando operaciones ACID y un manejo eficiente de versiones.
- soporta todos los tipos de analítica.

Esa capa transaccional se implementa mediante formatos como Delta Lake, Apache Iceberg y Apache Hudi, los cuales son conocidos como Open Table Format. En este material, nos enfocaremos en Delta Lake, una de las soluciones más utilizadas para construir un Data Lakehouse.


# Delta Lake y Python
Delta Lake es un formato de almacenamiento conocido como open table, o bien formato para lakehouse, que permite almacenar datos en un formato tabular, con la capacidad de realizar operaciones de lectura y escritura de datos de manera transaccional, es decir, que permite realizar operaciones de escritura y lectura de datos de manera atómica, lo que garantiza la integridad de los datos almacenados.

Otras alternativas a Delta Lake son Apache Iceberg y Apache Hudi.

Las tres tecnologías son open source, lo que permite que sean interoperables con varios lenguajes como Python, Rust, etc y otras tecnologías como Apache Spark, Apache Flink, etc.

En este notebook se mostrará cómo se puede trabajar con Delta Lake en Python, utilizando la librería [delta-io](https://delta-io.github.io/delta-rs) que permite trabajar con Delta Lake en Python.

Primero instalamos la libreria para trabajar con Delta Lake

Y luego instalamos pyarrow para un caso puntual que vere mas adelante

In [ ]:
!pip install deltalake
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 17.9 MB/s eta 0:00:00


In [ ]:
# Importamos las librerias y modulos necesarios
# write_deltalake: Modulo que permite escribir un DataFrame en un archivo Delta Lake
# DeltaTable: Modulo para manipular archivos Delta Lake

from deltalake import write_deltalake, DeltaTable
import pandas as pd
import pyarrow as pa
import os

Definimos dos diccionarios con datos de muestra.

Cada diccionario será utilizado para crear un DataFrame y posteriormente se escribirá en un archivo Delta.

¿Que notan de los datos de los diccionarios?

In [ ]:
data_1 = {
    "client_id": [1, 2, 3],
    "client_name": ["Mario Santos", "Emilio Ravenna", "Gabriel Medina"],
    "client_email": ["mario@gmail.com", "emilio@gmail.com", "gabriel@gmail.com"],
}

data_2 = {
    "client_id": [1, 4, 5],
    "client_name": ["Mario Santos", "Franco Milazzo", "Marcos Molero"],
    "client_email": ["mariosantos@gmail.com", "francomilazzo@gmail.com", "marcosmolero@gmail.com"]
}

In [ ]:
df_1 = pd.DataFrame(data_1)
df_1

,client_id,client_name,client_email
0,1,Mario Santos,mario@gmail.com
1,2,Emilio Ravenna,emilio@gmail.com
2,3,Gabriel Medina,gabriel@gmail.com


In [ ]:
df_2 = pd.DataFrame(data_2)
df_2

,client_id,client_name,client_email
0,1,Mario Santos,mariosantos@gmail.com
1,4,Franco Milazzo,francomilazzo@gmail.com
2,5,Marcos Molero,marcosmolero@gmail.com


## Modos de escritura o guardado
### 1. Modo 'error'

Creamos el 1er dataframe con el 1er diccionario `data1`.

Luego escribimos el dataframe como un archivo Delta.

Notese el parámetro mode='error' que indica que si el archivo ya existe, se lanzará un error. Es el valor por defecto.

In [ ]:
df_1 = pd.DataFrame(data_1)
write_deltalake(
    "data/clients", # ruta de guardado
    df_1, # dataframe con los datos
    mode="error") # valor por defecto

Creamos el 2do dataframe con el 2do diccionario `data2`.

Luego escribimos el dataframe como un archivo Delta, sobre la misma ruta que el archivo anterior.

Veremos que se lanza un error, ya que el archivo ya existe y no se puede sobreescribir.

In [ ]:
df_2 = pd.DataFrame(data_2)
write_deltalake(
    "data/clients",
    df_2,
    mode="error"
    ) # por defecto

DeltaError: Generic error: A table already exists at: /content/data/clients/

Por último, leemos el archivo Delta y lo mostramos como un DataFrame.
¿Que datos vamos a ver en el DataFrame?¿Cuántas filas tendrá?

In [ ]:
DeltaTable("data/clients").to_pandas()

,client_id,client_name,client_email
0,1,Mario Santos,mario@gmail.com
1,2,Emilio Ravenna,emilio@gmail.com
2,3,Gabriel Medina,gabriel@gmail.com


### 2. Modo 'ignore'
Vamos a guardar la primera tabla (dataframe `df_1`) en formato Delta Lake con este modo

In [ ]:
df_1 = pd.DataFrame(data_1)
write_deltalake(
    "data/clients_ignore",
    df_1,
    mode="ignore"
)

Ahora guardemos la 2da tabla, en la misma ruta. Que ocurrirá si usamos este método?

In [ ]:
df_2 = pd.DataFrame(data_2)
write_deltalake(
    "data/clients_ignore",
    df_2,
    mode="ignore"
)

Vamos a inspeccionar el contenido del directorio con nuestros datos en formato Delta Lake

In [ ]:
DeltaTable("data/clients_ignore").to_pandas()

,client_id,client_name,client_email
0,1,Mario Santos,mario@gmail.com
1,2,Emilio Ravenna,emilio@gmail.com
2,3,Gabriel Medina,gabriel@gmail.com


### 3. Modo 'overwrite'

Reutilizamos el 1er dataframe `df_1` y lo escribimos en un archivo Delta en otra ruta, pero esta vez con el parámetro mode='overwrite'.

In [ ]:
write_deltalake(
    "data/clients_overwrite",
    df_1,
    mode="overwrite"
    )

Usamos el 2do dataframe `df_2` y lo escribimos en la misma ruta que la celda anterior.

In [ ]:
write_deltalake(
    "data/clients_overwrite",
    df_2,
    mode="overwrite"
    )

Notese que no se lanza un error, ya que el archivo se sobreescribe. Entonces, si imprimimos el archivo Delta, veremos que solo contiene los datos del 2do dataframe.

In [ ]:
DeltaTable("data/clients_overwrite").to_pandas()

,client_id,client_name,client_email
0,1,Mario Santos,mariosantos@gmail.com
1,4,Franco Milazzo,francomilazzo@gmail.com
2,5,Marcos Molero,marcosmolero@gmail.com


Vamos a listar el contenido de la carpeta donde guardamos los datos.

In [ ]:
os.listdir("data/clients_overwrite")

['part-00001-095e57e5-2a4f-4b9b-8b37-bc300eb149e5-c000.snappy.parquet',
 '_delta_log',
 'part-00001-eb6dff2c-6404-4ccc-b207-eeb1d060c7f6-c000.snappy.parquet']

Los archivos .parquet son los archivos que contienen los datos de los DataFrames.
Ahora bien ¿por que aparecen dos archivos si realizamos una sobre-escritura?
Esto es porque Delta Lake guarda los datos en formato parquet, pero además guarda un archivo de metadatos que contiene la información de los cambios realizados en el archivo parquet, estos se ubican en el directorio `_delta_log`.

Al momento de realizar una lectura, se cargarán los datos de acuerdo a los metadatos registrados.

De hecho, si listamos el contenido de la carpeta `_delta_log`, veremos dos archivos, uno por cada operacion que realizamos.

In [ ]:
os.listdir("data/clients_overwrite/_delta_log")

['00000000000000000000.json', '00000000000000000001.json']

### 3. Modo 'append'
Ahora utilizaremos el modo 'append' para ir concatenando datos.

Los dataframes `df_1` y `df_2` se escribirán en un archivo Delta en una nueva ruta.

In [ ]:
write_deltalake(
    "data/clients_append_2",
    df_1,
    mode="error"
    )

write_deltalake(
    "data/clients_append_2",
    df_2,
    mode="append"
    )

¿Que datos veremos al leer el archivo Delta?

In [ ]:
DeltaTable("data/clients_append_2").to_pandas()

,client_id,client_name,client_email
0,1,Mario Santos,mariosantos@gmail.com
1,4,Franco Milazzo,francomilazzo@gmail.com
2,5,Marcos Molero,marcosmolero@gmail.com
3,1,Mario Santos,mario@gmail.com
4,2,Emilio Ravenna,emilio@gmail.com
5,3,Gabriel Medina,gabriel@gmail.com


Tambien podemos imprimir el dataset de forma ordenada.

In [ ]:
(
    DeltaTable("data/clients_append_2")
      .to_pandas()
      .sort_values("client_id")
)

,client_id,client_name,client_email
0,1,Mario Santos,mariosantos@gmail.com
3,1,Mario Santos,mario@gmail.com
6,1,Mario Santos,mariosantos@gmail.com
9,1,Mario Santos,mario@gmail.com
4,2,Emilio Ravenna,emilio@gmail.com
10,2,Emilio Ravenna,emilio@gmail.com
5,3,Gabriel Medina,gabriel@gmail.com
11,3,Gabriel Medina,gabriel@gmail.com
1,4,Franco Milazzo,francomilazzo@gmail.com
7,4,Franco Milazzo,francomilazzo@gmail.com


Cuidado con usar el modo "append", ya que si se ejecuta de forma descontrolada sin realizar validaciones podemos generar datos repetidos

In [ ]:
write_deltalake(
    "data/clients_append_2",
    df_1,
    mode="append"
    )

write_deltalake(
    "data/clients_append_2",
    df_2,
    mode="append"
    )

DeltaTable("data/clients_append_2").to_pandas().sort_values("client_id")

,client_id,client_name,client_email
0,1,Mario Santos,mariosantos@gmail.com
15,1,Mario Santos,mario@gmail.com
12,1,Mario Santos,mariosantos@gmail.com
3,1,Mario Santos,mario@gmail.com
6,1,Mario Santos,mariosantos@gmail.com
9,1,Mario Santos,mario@gmail.com
4,2,Emilio Ravenna,emilio@gmail.com
10,2,Emilio Ravenna,emilio@gmail.com
16,2,Emilio Ravenna,emilio@gmail.com
11,3,Gabriel Medina,gabriel@gmail.com


### 4. Operación MERGE/UPSERT
Por último, utilizaremos la operación MERGE para actualizar datos en un archivo Delta.

La operación MERGE permite realizar operaciones de actualización, inserción y eliminación de datos en un archivo Delta.

Mas info sobre la operación MERGE lo encontrarás en este [enlace](https://delta-io.github.io/delta-rs/api/delta_table/delta_table_merger/)

In [ ]:
# Primero se crea el archivo Delta con los datos de df_1
write_deltalake(
    "data/clients_merge",
    df_1
    )

Los nuevos datos que se quieran cargar con la operación MERGE deben ser una tabla de pyarrow. En la siguiente celda se muestra cómo se puede crear una tabla de pyarrow a partir de un dataframe.

Los datos actuales en el archivo Delta se cargan en la variable `actual_data`.

In [ ]:
# Los datos actuales ya estan en Delta lake, debemos leerlos
actual_data = DeltaTable("data/clients_merge")

# Creamos una tabla pyarrow, representando datos nuevos que estan en un dataframe
new_data = pa.Table.from_pandas(df_2)

In [ ]:
actual_data.to_pandas()

,client_id,client_name,client_email
0,1,Mario Santos,mario@gmail.com
1,2,Emilio Ravenna,emilio@gmail.com
2,3,Gabriel Medina,gabriel@gmail.com


A continuación se ve la aplicación de la operación MERGE, donde se actualizan los datos de la columna `client_mail`.

Si entre `actual_data` y `new_data` hay registros con el mismo `client_id`, se actualizará el `client_mail` con el valor que haya en `new_data`.

Si no hay registros con el mismo `client_id`, se insertará el registro de `new_data`.

In [ ]:
(
    actual_data.merge(
        source=new_data, # los datos nuevos a insertar o actualizar
        source_alias="src", # src -> new_data
        target_alias="tgt", # tgt -> actual_data
        predicate="src.client_id = tgt.client_id"
    )
    .when_matched_update(
        updates={
            "client_email": "src.client_email" # client mail de new_data
            #  "client_address": "src.client_address"
        }
    )
    .when_not_matched_insert_all()
    .execute()
)

{'num_source_rows': 3,
 'num_target_rows_inserted': 3,
 'num_target_rows_updated': 0,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 3,
 'num_target_files_scanned': 1,
 'num_target_files_skipped_during_scan': 0,
 'num_target_files_added': 1,
 'num_target_files_removed': 0,
 'execution_time_ms': 17,
 'scan_time_ms': 0,
 'rewrite_time_ms': 3}

In [ ]:
actual_data.to_pandas()

,client_id,client_name,client_email
0,4,Franco Milazzo,francomilazzo@gmail.com
1,5,Marcos Molero,marcosmolero@gmail.com
2,1,Mario Santos,mariosantos@gmail.com
3,2,Emilio Ravenna,emilio@gmail.com
4,3,Gabriel Medina,gabriel@gmail.com


In [ ]:
# Que pasa si volvemos a ejecutar el MERGE?
(
    actual_data.merge(
        source=new_data,
        source_alias="src",
        target_alias="tgt",
        predicate="src.client_id = tgt.client_id"
    )
    .when_matched_update(
        updates={
            "client_email": "src.client_email"
        }
    )
    .when_not_matched_insert_all()
    .execute()
)

{'num_source_rows': 3,
 'num_target_rows_inserted': 0,
 'num_target_rows_updated': 3,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 2,
 'num_output_rows': 5,
 'num_target_files_scanned': 2,
 'num_target_files_skipped_during_scan': 0,
 'num_target_files_added': 1,
 'num_target_files_removed': 2,
 'execution_time_ms': 19,
 'scan_time_ms': 0,
 'rewrite_time_ms': 4}

In [ ]:
actual_data.to_pandas()

,client_id,client_name,client_email
0,3,Gabriel Medina,gabriel@gmail.com
1,1,Mario Santos,mariosantos@gmail.com
2,2,Emilio Ravenna,emilio@gmail.com
3,5,Marcos Molero,marcosmolero@gmail.com
4,4,Franco Milazzo,francomilazzo@gmail.com


El resultado de ejecutar esta operacion imprime metadatos revelantes.

Vamos a mostrar el contenido del archivo Delta como un DataFrame. ¿Cuántos registros habrá?

In [ ]:
actual_data.to_pandas().sort_values("client_id")

,client_id,client_name,client_email
0,1,Mario Santos,mariosantos@gmail.com
1,2,Emilio Ravenna,emilio@gmail.com
2,3,Gabriel Medina,gabriel@gmail.com
3,4,Franco Milazzo,francomilazzo@gmail.com
4,5,Marcos Molero,marcosmolero@gmail.com


Si bien `actual_data` es una variable en memoria, al realizar la operación MERGE, se actualiza el archivo Delta.
Vamos a leer directamente del directorio donde se guardan los datos y mostrar el contenido como un DataFrame.

In [ ]:
DeltaTable("data/clients_merge").to_pandas().sort_values("client_id")

,client_id,client_name,client_email
0,1,Mario Santos,mariosantos@gmail.com
1,2,Emilio Ravenna,emilio@gmail.com
2,3,Gabriel Medina,gabriel@gmail.com
3,4,Franco Milazzo,francomilazzo@gmail.com
4,5,Marcos Molero,marcosmolero@gmail.com


Por último, para cerrar concluir la operación MERGE, podemos realizar una operación un poco mas sencilla.

Si al comparar `actual_data` y `new_data` hay registros con el mismo `client_id`, podemos omitir las coincidencias por `client_id` y solo insertar los registros que no coincidan.

In [ ]:
# Primero se crea el archivo Delta con los datos de df_1
write_deltalake(
    "data/clients_merge_only_insert",
    df_1
    )

Aplicamos la operacion MERGE como ya vimos.

¿Que ves de diferente en la operación?

In [ ]:
new_data = pa.Table.from_pandas(df_2)
actual_data = DeltaTable("data/clients_merge_only_insert")

(
    actual_data.merge(
        source=new_data,
        source_alias="src",
        target_alias="tgt",
        predicate="src.client_id = tgt.client_id"
    )
    .when_not_matched_insert_all()
    .execute()
)

{'num_source_rows': 3,
 'num_target_rows_inserted': 2,
 'num_target_rows_updated': 0,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 2,
 'num_target_files_scanned': 1,
 'num_target_files_skipped_during_scan': 0,
 'num_target_files_added': 1,
 'num_target_files_removed': 0,
 'execution_time_ms': 16,
 'scan_time_ms': 0,
 'rewrite_time_ms': 3}

In [ ]:
# Solo se insertan los datos nuevos
# No se realizan cambios en los datos existentes
actual_data.to_pandas().sort_values("client_id")

,client_id,client_name,client_email
2,1,Mario Santos,mario@gmail.com
3,2,Emilio Ravenna,emilio@gmail.com
4,3,Gabriel Medina,gabriel@gmail.com
1,4,Franco Milazzo,francomilazzo@gmail.com
0,5,Marcos Molero,marcosmolero@gmail.com


In [ ]:
# Que pasa si volvemos a ejecutar?
new_data = pa.Table.from_pandas(df_2)
actual_data = DeltaTable("data/clients_merge_only_insert")

(
    actual_data.merge(
        source=new_data,
        source_alias="src",
        target_alias="tgt",
        predicate="src.client_id = tgt.client_id"
    )
    # omitimos .when_matched_update
    .when_not_matched_insert_all()
    .execute()
)

{'num_source_rows': 3,
 'num_target_rows_inserted': 0,
 'num_target_rows_updated': 0,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 0,
 'num_target_files_scanned': 2,
 'num_target_files_skipped_during_scan': 0,
 'num_target_files_added': 0,
 'num_target_files_removed': 0,
 'execution_time_ms': 13,
 'scan_time_ms': 0,
 'rewrite_time_ms': 1}

## Constraints

Las constraints, o restricciones, son reglas que se pueden aplicar a los datos almacenados en un archivo Delta, para garantizar la integridad de los datos. Son una manera de asegurar que solo los datos que cumplen con ciertas reglas sean agregados al archivo Delta.

In [ ]:
#  Primero creamos el archivo Delta con los datos de df_1
df_1 = pd.DataFrame(data_1)
write_deltalake(
    "data/clients_constraints",
    df_1
    )

In [ ]:
# Vamos a leer alguna de las tablas Delta ya creadas
# y agregamos la restricción de que el campo client_id debe ser mayor a 0
dt = DeltaTable("data/clients_merge_only_insert")
dt.alter.add_constraint(
    # alias de la restriccion : restriccion/regla
    {"client_id_gt_0": "client_id > 0"}
)

In [ ]:
# Creamos un DataFrame con datos no válidos
data_no_valid = {
    "client_id": [9, 0],
    "client_name": ["anonimo", "Pablo Lampone"],
    "client_email": ["anonimo", "lampone@gmail.com"]
}
df_no_valid = pd.DataFrame(data_no_valid)
df_no_valid

,client_id,client_name,client_email
0,9,anonimo,anonimo
1,0,Pablo Lampone,lampone@gmail.com


In [ ]:
# Intentamos escribir los datos no válidos en la tabla Delta
write_deltalake(
    "data/clients_merge_only_insert",
    df_no_valid,
    mode="append",
    engine="rust"
    )

DeltaProtocolError: Invariant violations: ["Check or Invariant (client_id > 0) violated by value in row: [0, Pablo Lampone, lampone@gmail.com]"]

Si prestamos atención al error, podemos ver que un registro no cumple con una constraint.

In [ ]:
DeltaTable("data/clients_merge_only_insert").to_pandas()

,client_id,client_name,client_email
0,5,Marcos Molero,marcosmolero@gmail.com
1,4,Franco Milazzo,francomilazzo@gmail.com
2,1,Mario Santos,mario@gmail.com
3,2,Emilio Ravenna,emilio@gmail.com
4,3,Gabriel Medina,gabriel@gmail.com


## Particiones

Las particiones son una manera de organizar los datos, de manera que se puedan realizar operaciones de lectura y escritura de manera más eficiente.

Se puede particionar los datos por una o más columnas, de manera que los datos se guarden en directorios separados, de acuerdo a los valores de las columnas de partición.

In [ ]:
write_deltalake(
    "data/clients_partitioned",
    df_1,
    mode="append",
    partition_by="client_id"
)

In [ ]:
# Listamos los archivos en la carpeta de la tabla Delta, para ver las particiones creadas
# Cada carpeta tendrá archivos
os.listdir("data/clients_partitioned")

['client_id=3', 'client_id=1', 'client_id=2', '_delta_log']

In [ ]:
write_deltalake(
    "data/clients_partitioned",
    df_2,
    # mode="append",
    mode="overwrite",
    partition_by=["client_id"]
)

In [ ]:
os.listdir("data/clients_partitioned")

['client_id=3',
 'client_id=1',
 'client_id=2',
 'client_id=5',
 'client_id=4',
 '_delta_log']

In [ ]:
DeltaTable("data/clients_partitioned").to_pandas()

,client_id,client_name,client_email
0,4,Franco Milazzo,francomilazzo@gmail.com
1,5,Marcos Molero,marcosmolero@gmail.com
2,1,Mario Santos,mariosantos@gmail.com


Si usamos la opcion 'overwrite', las particiones originales permanecerán en el directorio pero no se verán reflejadas en el archivo Delta debido a la operación de sobreescritura.

## **Consideraciones importantes**

- Cuidado con el uso de 'append' ya que puede generar datos repetidos si es que no se agregan previamente restricciones. Se recomienda usar la operación MERGE para evitar este tipo de problemas.

- Si bien creamos tablas Delta lake a partir de un dataframe con datos, es posible crearlas como si fuesen una tabla de base de datos. Para ello, se puede utilizar el método [DeltaTable.create](https://delta-io.github.io/delta-rs/api/delta_table/#deltalake.DeltaTable.create).

## Conclusión

Delta Lake es una tecnología que permite almacenar datos de manera tabular y columnar, con la capacidad de realizar operaciones de lectura y escritura de datos de manera transaccional, lo que garantiza la integridad de los datos almacenados.

Si bien realizamos operacion de lectura y escritura, además de agregar restricciones, es posible también por ejemplo realizar operaciones de eliminación de datos, consultar versiones pasadas de los datos, etc.

Existen también operaciones de optimización de datos, como por ejemplo la operación VACUUM, que permite eliminar archivos antiguos y optimizar el espacio de almacenamiento, o la operación OPTIMIZE para compactar varios archivos pequeños en uno solo.

Las tabla delta lake permiten agregar metadatos relevantes, como autor de los datos, dominio al que pertenecen, descripción, etc. con el fin de catalogarlos y facilitar su gestión con otras herramientas.

Es una tecnología potente y versátil, que ofrece muchas posibilidades para trabajar con datos de manera eficiente.

Página oficial de Delta Lake: https://delta.io/